In [ ]:
#r "nuget: Microsoft.DotNet.Interactive, 1.0.0-beta.24568.1"
#r "nuget: Microsoft.AspNetCore.SignalR.Client, 8.0.0"

using Microsoft.DotNet.Interactive;
using Microsoft.AspNetCore.SignalR.Client;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.Events;

// Setup state
var kernel = Kernel.Root;
var isConnected = false;

// Setup SignalR connection with timeout
var connection = new HubConnectionBuilder()
    .WithUrl("https://localhost:65260/notebookhub")
    .WithAutomaticReconnect()
    .Build();

// Add blocking middleware with explicit error handling
kernel.AddMiddleware(async (KernelCommand command, KernelInvocationContext context, KernelPipelineContinuation next) =>
{
    if (!isConnected)
    {
        context.DisplayStandardError("❌ Not connected to remote kernel");
        return;
    }

    if (command is SubmitCode submitCode)
    {
        try 
        {
            var executionTask = connection.InvokeAsync<string>("ExecuteCode", new {
                Code = submitCode.Code,
                KernelName = kernel.Name
            });

            // Add timeout to detect unimplemented methods
            var timeoutTask = Task.Delay(2000);
            var completedTask = await Task.WhenAny(executionTask, timeoutTask);
            
            if (completedTask == timeoutTask)
            {
                context.DisplayStandardError("❌ Remote execution timeout - Method may not be implemented");
                return;
            }

            var result = await executionTask;
            if (string.IsNullOrEmpty(result))
            {
                context.DisplayStandardError("❌ No response from remote kernel");
            }
        }
        catch (Exception ex)
        {
            context.DisplayStandardError($"❌ Remote execution failed: {ex.Message}");
        }
        return; // Prevent local execution
    }
});

// Connect with explicit error logging
try
{
    await connection.StartAsync();
    
    // Test hub method existence
    try {
        await connection.InvokeAsync("Ping");
        Console.WriteLine("✅ Hub connection verified");
    }
    catch {
        Console.WriteLine("⚠️ Hub connected but methods may be unavailable");
    }
    
    await connection.SendAsync("RegisterNotebook", kernel.Name);
    isConnected = true;
    Console.WriteLine("✅ Connected to remote kernel");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Connection failed: {ex.Message}");
    isConnected = false;
}

Installed Packages Microsoft.AspNetCore.SignalR.Client, 8.0.0 Microsoft.DotNet.Interactive, 1.0.0-beta.24568.1

⚠️ Hub connected but methods may be unavailable
✅ Connected to remote kernel


In [ ]:
1+1

❌ Remote execution failed: Failed to invoke 'ExecuteCode' due to an error on the server. HubException: Method does not exist.

Installed Packages Microsoft.DotNet.Interactive, 1.0.0-beta.24568.1

Executing code: Console.WriteLine("Test execution");
Test execution
Command succeeded: SubmitCode: #r "nuget: Microsoft.DotNet.Interactive, 1.0.0-bet ...


In [ ]:
1+1